In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# matplotlib 한글 깨짐 해결
from matplotlib import rc

rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [3]:
data = pd.read_csv('../2023빅콘테스트_어드밴스드리그_예술의전당.csv')
data.head()

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,...,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211003,1550,...,0,초대권,1120,NaN,20210823.0,독주,리사이틀홀,90,15,N
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220917,1520,...,0,초대권,2283,20220911.0,20220911.0,독주,IBK챔버홀,80,15,N
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190630,1132,...,0,초대권,2798,NaN,20190701.0,독주,리사이틀홀,90,15,N
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,1615,...,0,일반,897,20220109.0,20220110.0,독주,콘서트홀,90,0,Y


# 기본 처리

- 콘서트홀 필터링  
- 중복 drop

In [4]:
# '콘서트홀'만 필터링
concert = data[data['place'] == '콘서트홀']

In [5]:
# 14개 중복 drop
concert.drop_duplicates(inplace=True)

/tmp/ipykernel_43751/3040057603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert.drop_duplicates(inplace=True)


In [6]:
# place 열 제거
concert.drop(['place'], axis=1, inplace=True)
# concert.columns

/tmp/ipykernel_43751/4244807966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert.drop(['place'], axis=1, inplace=True)


# 시간, 날짜 처리

In [7]:
# 분을 시간과 분으로 변환하는 함수
def convert_to_time(value):
    hours = value // 100  # 시간은 100으로 나눈 몫
    minutes = value % 100  # 분은 100으로 나눈 나머지
    return f"{hours:02d}:{minutes:02d}"  # 시간과 분을 두 자리 수로 표현

In [8]:
tran_time_series = pd.Series(concert['tran_time'])
play_time_series = pd.Series(concert['play_st_time'])

# 시리즈의 각 값을 시간 형식으로 변환
converted_times = tran_time_series.apply(convert_to_time)
converted_play_times = play_time_series.apply(convert_to_time)

# 변환된 결과 출력
print(converted_times[:5])
print(converted_play_times[:5])

0    15:12
4    16:15
5    11:45
6    14:59
7    09:54
Name: tran_time, dtype: object
0    20:00
4    19:30
5    20:00
6    17:00
7    20:00
Name: play_st_time, dtype: object


In [9]:
concert['tran_time'] = converted_times
concert['play_st_time'] = converted_play_times

concert.head(3)

/tmp/ipykernel_43751/3126354414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert['tran_time'] = converted_times
/tmp/ipykernel_43751/3126354414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert['play_st_time'] = converted_play_times


,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,running_time,intermission,member_yn
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,15:12,...,10000,2,일반,302,20220114.0,20220115.0,교향곡,120,15,Y
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,16:15,...,180000,0,일반,897,20220109.0,20220110.0,독주,90,0,Y
5,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,11:45,...,144000,2,블루회원 할인20%,528,20181119.0,20181119.0,교향곡,100,15,Y


In [10]:
concert[['play_date', 'tran_date', 'pre_open_date', 'open_date']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1096954 entries, 0 to 1920864
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   play_date      1096954 non-null  int64  
 1   tran_date      1096954 non-null  int64  
 2   pre_open_date  701312 non-null   float64
 3   open_date      1096954 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 41.8 MB


In [11]:
# 날짜 정보가 없는 경우(결측치) 0으로 채워줌
concert['pre_open_date'].fillna(0, inplace=True)
concert['open_date'].fillna(0, inplace=True)

/tmp/ipykernel_43751/73734063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert['pre_open_date'].fillna(0, inplace=True)
/tmp/ipykernel_43751/73734063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert['open_date'].fillna(0, inplace=True)


In [12]:
# 정수형으로 변환
concert[['pre_open_date', 'open_date']] = concert[['pre_open_date', 'open_date']].astype('int')

/tmp/ipykernel_43751/216437633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concert[['pre_open_date', 'open_date']] = concert[['pre_open_date', 'open_date']].astype('int')


In [13]:
concert[['play_date', 'tran_date', 'pre_open_date', 'open_date']]

,play_date,tran_date,pre_open_date,open_date
0,20220204,20220114,20220114,20220115
4,20220302,20220206,20220109,20220110
5,20190323,20181124,20181119,20181119
6,20191109,20191020,0,20191023
7,20190723,20190613,20190602,20190602
...,...,...,...,...
1920853,20210704,20210618,20210618,20210620
1920855,20230613,20230522,20230505,20230507
1920856,20201020,20201009,0,20200920
1920858,20200818,20200726,20200719,20200720


# 공연 code 생성

In [14]:
# 공연 날짜와 시간, 장르 기준으로 묶기 (고유한 공연 추출)
a = concert.groupby(['play_date', 'play_st_time'])['genre'].unique().to_frame().reset_index()
a['genre_cnt'] = a['genre'].apply(lambda x: len(x))
# 각 행 별로 공연이 하나씩 들어갔는지 확인
a[a['genre_cnt']>1]

,play_date,play_st_time,genre,genre_cnt


In [15]:
# 새로 코드 붙여주기
a['new_code'] = list(a.index)
a.drop(['genre', 'genre_cnt'], axis=1, inplace=True)
a

,play_date,play_st_time,new_code
0,20181125,17:00,0
1,20181208,17:00,1
2,20181218,20:00,2
3,20181220,20:00,3
4,20181223,15:00,4
...,...,...,...
746,20230702,17:00,746
747,20230704,19:30,747
748,20230705,19:30,748
749,20230706,11:00,749


In [16]:
# 기존 데이터에 붙여주기
concert = pd.merge(concert, a, on=['play_date', 'play_st_time'], how='left')
concert

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,15:12,...,2,일반,302,20220114,20220115,교향곡,120,15,Y,435
1,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,16:15,...,0,일반,897,20220109,20220110,독주,90,0,Y,449
2,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,11:45,...,2,블루회원 할인20%,528,20181119,20181119,교향곡,100,15,Y,31
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191020,14:59,...,0,초대권,44,0,20191023,교향곡,100,15,N,162
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,09:54,...,0,초대권,827,20190602,20190602,교향곡,120,20,N,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096949,60.0,F,무료,그린,NaN,NaN,NaN,NaN,20210618,15:09,...,2,일반,887,20210618,20210620,교향곡,110,20,Y,325
1096950,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20230522,17:29,...,0,일반,2825,20230505,20230507,교향곡,120,15,Y,732
1096951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201009,16:52,...,0,초대권,77,0,20200920,클래식,120,15,N,250
1096952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200726,16:55,...,0,기획사판매,850,20200719,20200720,합창,110,15,N,241


In [17]:
# 기존 performance code 열 제거
concert.drop(['performance_code'], axis=1, inplace=True)

# running_time=0의 경우 제거 
- (4481 rows, 3개 공연)

In [18]:
# concert[concert['running_time']==0]['new_code'].value_counts()

In [19]:
concert = concert[concert['running_time']!=0]
concert[concert['running_time']==0]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code


# 공연 및 좌석 필터링
- nan, 기타, 가족극, 재즈, 크로스오버 장르 제거  
- 전석초대인 공연 제거
- 공연진행석 제거 (관람객이 아니기 때문에 분석에서 제외)

In [20]:
# 공연진행석에 해당하는 데이터 개수 확인
concert[concert['discount_type']=='공연진행석'].shape

(12475, 23)

In [21]:
# 공연진행석 제거
concert = concert[concert['discount_type']!='공연진행석']
concert.shape

(1079998, 23)

In [22]:
# 전석초대 공연 코드 추출
agg_df = concert.groupby('new_code').agg({'price':'nunique', 'discount_type':'nunique'}).reset_index()
code_list = list(agg_df[(agg_df['price']==1) & (agg_df['discount_type']==1)]['new_code'])
# code_list

In [23]:
# 전석초대 공연 수
len(code_list)

60

In [25]:
concert[concert['new_code'].isin(code_list)].shape

(118280, 23)

In [26]:
concert.shape

(1079998, 23)

In [27]:
# 전석초대인 경우 제거
concert = concert[~concert['new_code'].isin(code_list)]

In [28]:
# 장르 nan인 경우 19911개
concert[concert['genre'].isna()].shape

(19911, 23)

In [29]:
concert.shape

(961718, 23)

In [30]:
# 장르가 nan인 경우 제거
concert.dropna(subset=['genre'], axis=0, inplace=True)

In [31]:
# 나머지 장르 제거
concert = concert[~concert['genre'].isin(['기타', '가족극', '재즈', '크로스오버'])]
concert['genre'].unique()

array(['교향곡', '독주', '클래식', '오페라', '합창', '성악', '실내악', '콘서트', '복합장르'],
      dtype=object)

In [32]:
concert[concert['genre'].isna()]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code


In [33]:
concert.shape

(939114, 23)

# 선예매여부

- 선예매일자가 예매시작일보다 이른 경우 제거

In [34]:
# pre_open_date가 open_date보다 늦은 공연 2개 (교향곡에서 2개)

concert[concert['pre_open_date']>concert['open_date']]['new_code'].unique()
# merge_df[merge_df['pre_open_date']>merge_df['open_date']]['discount_cat'].unique()
# merge_df[merge_df['pre_open_date']>merge_df['open_date']]['genre'].unique()

array([630, 417])

In [35]:
concert[concert['pre_open_date']>concert['open_date']].shape

(2935, 23)

In [36]:
concert.shape

(939114, 23)

In [37]:
# 선예매일자가 예매시작일보다 이른 경우 제거
concert = concert[~concert['new_code'].isin([627, 414])]
concert.shape

(936784, 23)

- (예매시작일, 공연날짜) (공연날짜, 예매 거래일자) (공연날짜, 선예매시작일) 간 관계는 모두 정상인 것으로 파악

In [38]:
# 예매시작일이 공연날짜보다 늦은 경우는 없음
concert[concert['open_date']>concert['play_date']]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code


In [39]:
# 공연날짜보다 예매 거래일자가 늦은 경우는 없음
concert[concert['play_date']<concert['tran_date']]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code


In [40]:
# 공연날짜보다 선예매시작일이 늦은 경우는 없음
concert[concert['play_date']<concert['pre_open_date']]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,price,ticket_cancel,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code


- 선예매시작일보다 예매거래가 먼저 일어나는 경우 존재  
=> 초대권이나 기획사 초대로 예매된 내역의 경우 선예매여부 인정  
=> 그 외의 경우 미인정

In [41]:
# 선예매시작일보다 예매거래가 먼저 일어나는 경우 존재...
concert[concert['tran_date']<concert['pre_open_date']][['tran_date', 'tran_time', 'pre_open_date', 'open_date','seat', 'member_yn', 'new_code']][:5]

,tran_date,tran_time,pre_open_date,open_date,seat,member_yn,new_code
8,20221030,16:45,20221105,20221030,2층 D블록8열 3,Y,630
52,20211118,18:27,20211119,20211119,3층 F블록6열 6,N,405
227,20220909,16:01,20220916,20220917,1층 C블록11열 5,Y,566
330,20220613,16:41,20220616,20220616,2층 A블록3열 10,N,527
463,20190726,11:02,20190729,20190802,1층 C블록4열 2,Y,127


In [42]:
concert['key'] = concert.index

In [43]:
# (1) 선예매시작일보다 예매거래가 먼저 일어나는 경우 추출
error_df = concert[concert['tran_date']<concert['pre_open_date']][['key', 'tran_date', 'pre_open_date', 'open_date', 'discount_type']]
print(error_df.head())
print()
print('shape:', error_df.shape)

     key  tran_date  pre_open_date  open_date discount_type
8      8   20221030       20221105   20221030            일반
52    52   20211118       20211119   20211119         기획사판매
227  227   20220909       20220916   20220917    그린회원 할인10%
330  330   20220613       20220616   20220616           초대권
463  463   20190726       20190729   20190802    골드회원 할인30%

shape: (6199, 5)


In [44]:
# (1-1) 선예매시작일보다 예매거래가 먼저 일어나는 경우 중 초대권과 기획사판매 표인 경우 추출
given_df = error_df[error_df['discount_type'].isin(['초대권', '기획사판매'])]
given_df['discount_type'].unique()

array(['기획사판매', '초대권'], dtype=object)

In [45]:
given_df.shape

(2793, 5)

In [46]:
# (1-2) 선예매시작일보다 예매거래가 먼저 일어나는 경우 중 초대권과 기획사판매 표가 아닌 경우 추출
rest_df = error_df[~error_df['discount_type'].isin(['초대권', '기획사판매'])]
rest_df.shape

(3406, 5)

In [47]:
# (**1**) 선예매시작일보다 예매거래가 먼저 일어나는 경우에 초대권/기획사초대인 경우 선예매여부 인정, 그 외에는 미인정
given_df['pre_ticketing'] = 1
rest_df['pre_ticketing'] = 0

print(given_df['pre_ticketing'].isna().sum())
print(rest_df['pre_ticketing'].isna().sum())

0
0


/tmp/ipykernel_43751/1504180285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  given_df['pre_ticketing'] = 1
/tmp/ipykernel_43751/1504180285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_df['pre_ticketing'] = 0


In [48]:
concert.shape

(936784, 24)

In [49]:
# (2) 선예매시작일과 예매거래 관계가 정상인 경우 추출
error_key = list(given_df['key'].values) + list(rest_df['key'].values)
no_error_df = concert[~concert['key'].isin(error_key)]
no_error_df.shape

(930585, 24)

In [50]:
# (2-1) 정상의 경우 중 선예매가 이루어진 경우에 선예매여부=1 부여
no_and_pre = no_error_df[(no_error_df['pre_open_date']!=0) & (no_error_df['tran_date']>no_error_df['pre_open_date']) & (no_error_df['tran_date']<no_error_df['open_date'])][['key', 'tran_date', 'pre_open_date', 'open_date', 'discount_type']]
no_and_pre['pre_ticketing'] = 1

In [51]:
# (2-2) 정상의 경우 중 선예매가 이루어지지 않은 경우에 선예매여부=0 부여
no_and_pre_key = list(no_and_pre['key'])
not_pre_df = no_error_df[~no_error_df['key'].isin(no_and_pre_key)][['key', 'tran_date', 'pre_open_date', 'open_date', 'discount_type']]
not_pre_df['pre_ticketing'] = 0

In [52]:
print(given_df.shape)
print(rest_df.shape)
print(no_and_pre.shape)
print(not_pre_df.shape)

(2793, 6)
(3406, 6)
(6781, 6)
(923804, 6)


In [53]:
pre_ticket_result_df = pd.concat([given_df, rest_df, no_and_pre, not_pre_df])
pre_ticket_result_df = pre_ticket_result_df[['key', 'pre_ticketing']]
pre_ticket_result_df

,key,pre_ticketing
52,52,1
330,330,1
1324,1324,1
1526,1526,1
1529,1529,1
...,...,...
1096949,1096949,0
1096950,1096950,0
1096951,1096951,0
1096952,1096952,0


In [54]:
pre_ticket_result_df['pre_ticketing'].isna().sum()

0

In [55]:
# 선예매여부 원래 데이터에 merge
concert = pd.merge(concert, pre_ticket_result_df, on='key', how='left')
concert.head()

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,discount_type,pre_open_date,open_date,genre,running_time,intermission,member_yn,new_code,key,pre_ticketing
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,15:12,...,일반,20220114,20220115,교향곡,120,15,Y,435,0,0
1,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,16:15,...,일반,20220109,20220110,독주,90,0,Y,449,1,0
2,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,11:45,...,블루회원 할인20%,20181119,20181119,교향곡,100,15,Y,31,2,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,09:54,...,초대권,20190602,20190602,교향곡,120,20,N,100,4,0
4,NaN,F,블루,무료,NaN,NaN,NaN,NaN,20190703,09:08,...,일반,0,20190413,클래식,100,15,N,99,5,0


In [56]:
concert.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6', 'tran_date', 'tran_time', 'play_date',
       'play_st_time', 'seat', 'price', 'ticket_cancel', 'discount_type',
       'pre_open_date', 'open_date', 'genre', 'running_time', 'intermission',
       'member_yn', 'new_code', 'key', 'pre_ticketing'],
      dtype='object')

# 멤버십 정리

## 멤버십 재정리 
- membership_1 등의 형태로 들어 있는 것을 멤버십 종류별 칼럼으로

In [57]:
# 멤버십 정리 (하나의 열로 몰아넣기)
cols = ['membership_type_1', 'membership_type_2', 'membership_type_3', 'membership_type_4', 'membership_type_5', 'membership_type_6']
concert['membership_all'] =concert[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
concert['membership_all']

0            골드 무료 그린 nan nan nan
1          무료 nan nan nan nan nan
2           블루 무료 nan nan nan nan
3         nan nan nan nan nan nan
4           블루 무료 nan nan nan nan
                   ...           
936779      무료 그린 nan nan nan nan
936780     무료 nan nan nan nan nan
936781    nan nan nan nan nan nan
936782    nan nan nan nan nan nan
936783    무료 싹틔우미 nan nan nan nan
Name: membership_all, Length: 936784, dtype: object

In [58]:
# nan값 등 정리
def merge_memberships(x):
    x = x.replace('nan', '')
    if x.isspace():
        x = re.sub(r"\s+", " ", x)
        x = x.replace(' ', 'nan')
    
    x = x.lstrip().rstrip()

    x = x.replace(' ', '|')

    return x

In [59]:
concert['membership_all'] = concert['membership_all'].apply(lambda x: merge_memberships(x))
concert[['membership_type_1', 'membership_type_2', 'membership_type_3', 'membership_type_4', 'membership_type_5', 'membership_type_6', 'membership_all']]

,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,membership_all
0,골드,무료,그린,NaN,NaN,NaN,골드|무료|그린
1,무료,NaN,NaN,NaN,NaN,NaN,무료
2,블루,무료,NaN,NaN,NaN,NaN,블루|무료
3,NaN,NaN,NaN,NaN,NaN,NaN,nan
4,블루,무료,NaN,NaN,NaN,NaN,블루|무료
...,...,...,...,...,...,...,...
936779,무료,그린,NaN,NaN,NaN,NaN,무료|그린
936780,무료,NaN,NaN,NaN,NaN,NaN,무료
936781,NaN,NaN,NaN,NaN,NaN,NaN,nan
936782,NaN,NaN,NaN,NaN,NaN,NaN,nan


In [60]:
# 멤버십 타입을 칼럼으로 만들기
membership_type = [x.split('|') for x in concert['membership_all'].values]
membership_type[:5]

[['골드', '무료', '그린'], ['무료'], ['블루', '무료'], ['nan'], ['블루', '무료']]

In [61]:
import itertools

mems = list(set(list(itertools.chain(*membership_type))))
print(len(membership_type))
print(len(mems))
print(mems)

936784
7
['싹틔우미', '무료', 'nan', '골드', '그린', '블루', '노블']


In [62]:
membership_df = pd.DataFrame(columns=mems, index=concert.index)
membership_df.head()

,싹틔우미,무료,nan,골드,그린,블루,노블
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# ct = 0
for i, row in membership_df.iterrows():
    idx = row.name
    list_of_membership = concert.loc[idx]['membership_all'].split('|')
    
    membership_df.loc[idx][list_of_membership] = 1

In [64]:
membership_df = membership_df.fillna(0)
membership_df['num_memberships'] = membership_df.sum(axis=1)
print(membership_df.shape)
print(membership_df)

(936784, 8)
        싹틔우미  무료  nan  골드  그린  블루  노블  num_memberships
0          0   1    0   1   1   0   0                3
1          0   1    0   0   0   0   0                1
2          0   1    0   0   0   1   0                2
3          0   0    1   0   0   0   0                1
4          0   1    0   0   0   1   0                2
...      ...  ..  ...  ..  ..  ..  ..              ...
936779     0   1    0   0   1   0   0                2
936780     0   1    0   0   0   0   0                1
936781     0   0    1   0   0   0   0                1
936782     0   0    1   0   0   0   0                1
936783     1   1    0   0   0   0   0                2

[936784 rows x 8 columns]


In [65]:
membership_df = concert['membership_all'].str.get_dummies(sep='|')
membership_df.head()

,nan,골드,그린,노블,무료,블루,싹틔우미
0,0,1,1,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0
3,1,0,0,0,0,0,0
4,0,0,0,0,1,1,0


In [66]:
concert_df = pd.concat([concert, membership_df], axis=1)
concert_df.head()

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,key,pre_ticketing,membership_all,nan,골드,그린,노블,무료,블루,싹틔우미
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,15:12,...,0,0,골드|무료|그린,0,1,1,0,1,0,0
1,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,16:15,...,1,0,무료,0,0,0,0,1,0,0
2,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,11:45,...,2,0,블루|무료,0,0,0,0,1,1,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,09:54,...,4,0,nan,1,0,0,0,0,0,0
4,NaN,F,블루,무료,NaN,NaN,NaN,NaN,20190703,09:08,...,5,0,블루|무료,0,0,0,0,1,1,0


In [67]:
concert_df.drop(['nan', 'membership_type_1', 'membership_type_2', 'membership_type_3', 'membership_type_4', 'membership_type_5', 'membership_type_6', 'membership_all'], axis=1, inplace=True)

In [68]:
concert_df.rename(columns={'블루':'blue', '그린':'green', '골드':'gold', '무료':'general'}, inplace=True)

## 무료 멤버십 이상치 확인

- 싹틔우미, 노블 멤버십과 연령 확인

In [69]:
# 싹틔우미를 가지고 있지만 10-20대가 아닌 경우
concert_df[(concert_df['싹틔우미']==1) & (concert_df['age']!=20.0) & (concert_df['age']!=10.0)].shape

(18357, 25)

In [70]:
# 노블을 가지고 있는데 70-80대가 아닌 경우
concert_df[(concert_df['노블']==1) & (concert_df['age']!=70.0) & (concert_df['age']!=80.0)].shape

(464, 25)

In [71]:
# 싹틔우미 오류 정리 함수
def cleanse_age(x):
    if x['싹틔우미'] == 1 and x['age'] != 20.0 and x['age'] != 10.0:
        return 0
    else:
        return x['싹틔우미']

In [72]:
concert_df['sac'] = concert_df.apply(lambda x: cleanse_age(x), axis=1)

In [73]:
# 싹틔우미 잘못 기재된 경우 제거
concert_df[(concert_df['sac']==1) & (concert_df['age']!=20.0) & (concert_df['age']!=10.0)]

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,new_code,key,pre_ticketing,gold,green,노블,general,blue,싹틔우미,sac


In [74]:
# 노블 오류 정리
def cleanse_noble(x):
    if x['노블'] == 1 and x['age'] != 70.0 and x['age'] != 80.0:
        return 0
    else:
        return x['노블']

In [75]:
concert_df['noble'] = concert_df.apply(lambda x: cleanse_noble(x), axis=1)
concert_df.head(3)

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,key,pre_ticketing,gold,green,노블,general,blue,싹틔우미,sac,noble
0,50.0,F,20220114,15:12,20220204,20:00,3층 BOX9 10,10000,2,일반,...,0,0,1,1,0,1,0,0,0,0
1,50.0,M,20220206,16:15,20220302,19:30,1층 B블록12열 7,180000,0,일반,...,1,0,0,0,0,1,0,0,0,0
2,30.0,F,20181124,11:45,20190323,20:00,1층 A블록2열 1,144000,2,블루회원 할인20%,...,2,0,0,0,0,1,1,0,0,0


In [76]:
concert_df[(concert_df['noble']==1) & (concert_df['age']!=70.0) & (concert_df['age']!=80.0)]

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,key,pre_ticketing,gold,green,노블,general,blue,싹틔우미,sac,noble


In [77]:
# 기존 '싹틔우미'와 '노블' 칼럼 제거
concert_df.drop(['노블', '싹틔우미'], axis=1, inplace=True)

- 회원이 아니지만 회원 정보 있는 경우 확인

In [78]:
concert_df[concert_df['member_yn']=='N']['gender'].unique()

array([nan, 'F', 'M'], dtype=object)

In [79]:
# 회원이 아니지만 성별 정보가 있는 경우(=탈퇴회원) 다시 회원으로 변경
def change_member_yn(x):
    if x['member_yn']=='N' and (x['gender']=='F' or x['gender']=='M'):
        return 'Y'
    else:
        return x['member_yn']

In [80]:
concert_df['member_yn'] = concert_df.apply(lambda x: change_member_yn(x), axis=1)
concert_df[concert_df['member_yn']=='N']['gender'].unique()

array([nan], dtype=object)

In [81]:
concert_df[(concert_df['member_yn']=='Y') & (concert_df['general']==0)]

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,member_yn,new_code,key,pre_ticketing,gold,green,general,blue,sac,noble


In [82]:
# # member_yn=='Y'와 무료회원 데이터 수 일치
# concert_df[concert_df['member_yn']=='Y']
# concert_df[concert_df['general']==1]

# 할인종류와 할인률

In [83]:
# merge_df['key'] = merge_df.index
# 할인 받지 않은 예매 건
no_discount = concert_df[~concert_df.discount_type.str.contains('%',na=False)]
no_discount.discount_type.unique()

array(['일반', '초대권', '기획사판매', '한화초대석', '공연예매권', '기획사', '신세계석', '정기회원',
       '기업석', '홍보진행', 'KT석', '문화햇살', '차액', '당일할인티켓', '일만원의 나눔', '당일할인티켓_',
       '기획사할인', '중앙일보 JTBC 초대권', '한화생명', '초/중/고등학생 할인(본인만)', 'BC카드',
       '초/중/고등학생 할인', '국립심포니', '중앙일보 JTBC', '특판B', '후원회석',
       '노르웨이의 숲 도서 인증 할인', '신한은행', '도서 인증 S', '이만원의 기쁨', '골드회원+음반 패키지',
       '홍보마케팅', '블랙프라이데이 이벤트', '특판D', 'A석+호두까기인형 패키지', '수험생 할인(동반1인)',
       '도서 인증 A', '삼만원의 행복', '하비에르 국제학교 학부모, 직원', '2+1 특별할인', '홍보진행석',
       '특판A', '특별할인(2+1)', '골드회원 할인+음반패키지', '오만원의 사랑', '하비에르 국제학교 재학생',
       '골드회원+음반패키지', 'S석+호두까기인형 패키지', '초/중/고/대학생(본인)+음반패키지', 'Art+초대',
       '기획사(특별할인)', '싹틔우미 할인'], dtype=object)

In [84]:
# 할인 받은 예매 건
discount = concert_df[concert_df.discount_type.str.contains('%')]
discount.discount_type.unique()

array(['블루회원 할인20%', 'K-lang 멤버십 회원(1인2매)20%', '골드회원 할인30%', '싹틔우미 할인40%',
       '라흐마니노프탄생150주년(8매/3.27까지)20%', '골드회원 할인10%', '장애인/국가유공자 할인50%',
       '싹딜가(골드)50%', '그린회원 할인5%', '초,중,고,대학생(본인만)30%', '조기예매 할인40%',
       '그린회원 할인20%', '단체100인이상50%', '초/중/고등학생 할인(본인만)30%', '골드회원 할인25%',
       '조기예매 할인(1인8매/3월11일까지)30%', '골드회원 할인5%', '만65세 이상(본인만)50%',
       '그린회원 할인15%', '직장인 할인(1인2매)20%', '블루회원 할인10%',
       '초/중/고/대학생 할인(본인만)50%', '문화릴레이(1인2매/공연유료티켓제시)10%', '조기예매 할인10%',
       'K-Lang 할인(1인2매)20%', '골드회원 할인40%', '초/중/고등학생 할인(본인만)20%',
       '조기예매 할인(1인2매/~10월31일까지)30%', '초/중/고/대학생 할인30%',
       '의사상자(동반1인/신분증지참)50%', '초,중,고,대학생(본인만)50%',
       '조기예매할인(1인8매/~6.30까지)30%', '골드회원 할인20%', '그린회원 할인10%', '블루회원 할인5%',
       '스탭 할인30%', 'KOSYMI  유료회원(1인4매)40%', '청소년(만 24세 이하/본인만)30%',
       '얼리버드 할인(1인4매/10월31일까지)10%', '패밀리할인(3매이상/1인4매)15%', '싹딜가(그린)60%',
       '조기예매 할인(1인8매/9월16일까지)30%', '청소년(만7세~만24세/신분증지참)20%',
       '실버할인(만65세 이상/본인만)50%', '조기예매할인(1인2매/~9월8일까지)30%',
       '얼리버

In [85]:
discount = discount.reset_index(drop=True)
discount

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,member_yn,new_code,key,pre_ticketing,gold,green,general,blue,sac,noble
0,30.0,F,20181124,11:45,20190323,20:00,1층 A블록2열 1,144000,2,블루회원 할인20%,...,Y,31,2,0,0,0,1,1,0,0
1,50.0,F,20191005,14:17,20191201,17:00,3층 BOX9 5,24000,0,K-lang 멤버십 회원(1인2매)20%,...,Y,174,6,0,0,0,1,0,0,0
2,70.0,M,20191220,11:06,20200211,19:30,2층 BOX2 2,42000,0,골드회원 할인30%,...,Y,208,11,0,1,0,1,0,0,0
3,20.0,F,20230327,16:01,20230523,19:30,1층 B블록21열 9,66000,0,싹틔우미 할인40%,...,Y,715,16,0,0,0,1,0,1,0
4,50.0,F,20230317,13:48,20230523,19:30,1층 B블록17열 4,88000,0,라흐마니노프탄생150주년(8매/3.27까지)20%,...,Y,715,17,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319467,50.0,F,20181209,14:07,20190224,17:00,2층 E블록2열 1,38000,2,골드회원 할인30%,...,Y,17,1096929,0,1,0,1,1,0,0
319468,30.0,F,20191206,11:02,20191222,17:00,2층 A블록5열 5,40000,0,싹딜가(그린)50%,...,Y,189,1096936,0,0,1,1,0,0,0
319469,50.0,M,20210917,15:44,20211009,17:00,2층 B블록3열 11,35000,2,골드회원 할인30%,...,Y,380,1096938,0,1,0,1,0,0,0
319470,50.0,F,20220410,14:01,20220626,17:00,1층 C블록5열 9,142000,2,그린회원 할인5%,...,Y,507,1096941,0,0,1,1,0,0,0


In [86]:
import re

def extract_percentage(s):
    match = re.search(r'(\d+%)', s)
    if match:
        return match.group(1)
    return None


discount['discount_rate'] = discount['discount_type'].apply(extract_percentage)

In [87]:
d = discount[['discount_type','discount_rate']]
d

,discount_type,discount_rate
0,블루회원 할인20%,20%
1,K-lang 멤버십 회원(1인2매)20%,20%
2,골드회원 할인30%,30%
3,싹틔우미 할인40%,40%
4,라흐마니노프탄생150주년(8매/3.27까지)20%,20%
...,...,...
319467,골드회원 할인30%,30%
319468,싹딜가(그린)50%,50%
319469,골드회원 할인30%,30%
319470,그린회원 할인5%,5%


In [88]:
def percentage_to_int(s):
    return int(s.strip('%'))

discount['discount_rate'] = discount['discount_rate'].apply(percentage_to_int)
discount.discount_rate.unique()

array([20, 30, 40, 10, 50,  5, 25, 15, 60, 35, 12, 75, 80])

In [89]:
# discount['정가'] = discount['할인율']

def calculate_original_price(row):
    discounted_price = row['price']
    rate = row['discount_rate']
    
    original_price = int(discounted_price) / (1 - int(rate) / 100)
    return int(original_price)

# apply 함수를 사용하여 새로운 열 추가
discount['real_price'] = discount.apply(calculate_original_price, axis=1)

In [90]:
discount[['price','discount_type','discount_rate','real_price']]

,price,discount_type,discount_rate,real_price
0,144000,블루회원 할인20%,20,180000
1,24000,K-lang 멤버십 회원(1인2매)20%,20,30000
2,42000,골드회원 할인30%,30,60000
3,66000,싹틔우미 할인40%,40,110000
4,88000,라흐마니노프탄생150주년(8매/3.27까지)20%,20,110000
...,...,...,...,...
319467,38000,골드회원 할인30%,30,54285
319468,40000,싹딜가(그린)50%,50,80000
319469,35000,골드회원 할인30%,30,50000
319470,142000,그린회원 할인5%,5,149473


In [91]:
main = discount[['key', 'discount_rate', 'real_price']]
main

,key,discount_rate,real_price
0,2,20,180000
1,6,20,30000
2,11,30,60000
3,16,40,110000
4,17,20,110000
...,...,...,...
319467,1096929,30,54285
319468,1096936,50,80000
319469,1096938,30,50000
319470,1096941,5,149473


In [92]:
merge_df = pd.merge(concert_df, main, on='key', how='left')
merge_df

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,key,pre_ticketing,gold,green,general,blue,sac,noble,discount_rate,real_price
0,50.0,F,20220114,15:12,20220204,20:00,3층 BOX9 10,10000,2,일반,...,0,0,1,1,1,0,0,0,NaN,NaN
1,50.0,M,20220206,16:15,20220302,19:30,1층 B블록12열 7,180000,0,일반,...,1,0,0,0,1,0,0,0,NaN,NaN
2,30.0,F,20181124,11:45,20190323,20:00,1층 A블록2열 1,144000,2,블루회원 할인20%,...,2,0,0,0,1,1,0,0,20.0,180000.0
3,NaN,NaN,20190613,09:54,20190723,20:00,2층 D블록8열 4,0,0,초대권,...,4,0,0,0,0,0,0,0,NaN,NaN
4,NaN,F,20190703,09:08,20190721,17:00,1층 C블록17열 3,75000,0,일반,...,5,0,0,0,1,1,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936779,60.0,F,20210618,15:09,20210704,15:00,1층 A블록2열 2,90000,2,일반,...,1096949,0,0,1,1,0,0,0,NaN,NaN
936780,50.0,M,20230522,17:29,20230613,17:00,3층 A블록6열 4,10000,0,일반,...,1096950,0,0,0,1,0,0,0,NaN,NaN
936781,NaN,NaN,20201009,16:52,20201020,19:30,1층 D블록16열 12,0,0,초대권,...,1096951,0,0,0,0,0,0,0,NaN,NaN
936782,NaN,NaN,20200726,16:55,20200818,19:30,1층 D블록20열 8,0,0,기획사판매,...,1096952,0,0,0,0,0,0,0,NaN,NaN


In [93]:
merge_df['real_price'].fillna(merge_df['price'], inplace=True)

In [94]:
merge_df['discount_rate'].fillna(0, inplace=True)

In [95]:
merge_df[['price','discount_type','real_price']]

,price,discount_type,real_price
0,10000,일반,10000.0
1,180000,일반,180000.0
2,144000,블루회원 할인20%,180000.0
3,0,초대권,0.0
4,75000,일반,75000.0
...,...,...,...
936779,90000,일반,90000.0
936780,10000,일반,10000.0
936781,0,초대권,0.0
936782,0,기획사판매,0.0


In [96]:
def map_category(x):
    if any(t in x for t in ['패밀리', '가정의', '가족', '아이랑노랑']):
        return '가족'
    elif any(t in x for t in ['장애인', '의상자', '국가유공자', '경기도민','유족', 
                              '교직원','임산부','보건의료인','다둥이','예비군',
                              '문화누리','군인','의사상자','강남구','지방 콘서트']):
        return '국가'
    elif any(t in x for t in ['만65세','실버','경로우대','65세','노약자', '70세']):
        return '노인'
    elif any(t in x for t in ['초대권','lang','기획사','초대석','공연예매권','Lang','신세계석','스탭','기업',
                              '홍보진행','KOSYMI','KT','주보','카톡친구','시향직원','단원','한화','유료회원',
                              '기획사','멜론','국립심포니','30주년','국립','유튜브','중앙일보','예술인',
                              '클립서비스','후원','서포터즈','Point','KBS','합창마니아','초대','스텝','출연진',
                              '출연자','참석자','연주자','단원','멜론','국립','카톡친구','정기회원','문화예술계','카톡',
                              '또모','라벨라오페라단','홍보마케팅','학부모','코심콘서트강의','호두까기인형']):
        return '기업 및 관계자'
    elif any(t in x for t in ['싹틔우미', '노블회원', '블루','골드','그린','싹딜가','문화햇살', '공연진행석','후원회원']):
        return '예술의 전당'
    elif any(t in x for t in ['조기예매', '얼리버드', '당일티켓', '재관람', '티켓소지자', '타임세일', '패키지 추가구매', 
                              '문화릴레이','추가구매','재구매','기구매자','재예매','당일할인티켓','시즌패키지','상반기 패키지','소지자','보고 또 보고']):
        return '공연'
    elif any(t in x for t in ['초/중/고','대학생','고등학생','청소년','학생','만 29세','수험생','재학생','만29세','청년패스','초,중,고', '음대생', '전공자']):
        return '학생'
    elif any(t in x for t in ['라흐마니노프','단체','백신','직장인','우리카드','신한','카드','맞이특별',
                              '30주년','기부','인증''2+1','웰컴','일만원','다이내믹','마니아','베토벤',
                              '이만원','삼만원','오만원','발렌타인','박규희','프라이데이','우리랑','차액','GOOD',
                              '쿠폰','특별','화이트데이','코코프렌즈','호랑이띠','특판','토끼띠','노르웨이의 숲 도서 인증 할인',
                              '카카오톡','추석','가을','여름방학','도서 인증','도서인증']):
        return '기타 및 이벤트'
    elif x == '일반':
        return '일반'

In [97]:
merge_df['discount_cat'] = merge_df['discount_type'].apply(lambda x: map_category(x))
merge_df.head(2)

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,pre_ticketing,gold,green,general,blue,sac,noble,discount_rate,real_price,discount_cat
0,50.0,F,20220114,15:12,20220204,20:00,3층 BOX9 10,10000,2,일반,...,0,1,1,1,0,0,0,0.0,10000.0,일반
1,50.0,M,20220206,16:15,20220302,19:30,1층 B블록12열 7,180000,0,일반,...,0,0,0,1,0,0,0,0.0,180000.0,일반


# 회원 정보와 할인종류 안 맞는 경우 확인

In [98]:
# 무료+유료 멤버십 개수
merge_df['all_mem_cnt'] = merge_df[['green', 'blue', 'gold', 'sac', 'noble', 'general']].sum(axis=1)
merge_df.head(1)

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,gold,green,general,blue,sac,noble,discount_rate,real_price,discount_cat,all_mem_cnt
0,50.0,F,20220114,15:12,20220204,20:00,3층 BOX9 10,10000,2,일반,...,1,1,1,0,0,0,0.0,10000.0,일반,3


In [99]:
# 초대권인데 회원정보가 있는 경우
not_inv_code = merge_df[(merge_df['discount_type']=='초대권') & (merge_df['all_mem_cnt']>0)]['key']
# 싹틔우미 할인을 받았으나 나이+회원여부 기준으로 싹틔우미 회원일 수 없는 경우
not_sac_code = merge_df[(merge_df['discount_type'].str.contains('싹틔우미')) & (merge_df['sac']==0)]['key']
# 노블 할인을 받았으나 나이+회원여부 기준으로 노블 회원일 수 없는 경우
not_noble_code = merge_df[(merge_df['discount_type'].str.contains('노블')) & (merge_df['noble']==0)]['key']

del_codes = list(not_inv_code) + list(not_sac_code) + list(not_noble_code)
len(del_codes)

355

In [101]:
# 위 3개 케이스 제거
merge_df = merge_df[~merge_df['key'].isin(del_codes)]
merge_df.shape

(936429, 29)

In [106]:
# 유료 멤버십 관련 할인 받았지만 멤버십이 없다고 되어 있는 경우

# 블루 회원
blue_error = merge_df[merge_df['discount_type'].str.contains('블루')]
blue_change = blue_error[(blue_error['member_yn']=='Y') & (blue_error['blue']==0)]['key']
blue_remove = blue_error[(blue_error['member_yn']=='N') & (blue_error['blue']==0)]['key']

# 그린 회원
green_error = merge_df[merge_df['discount_type'].str.contains('그린')]
green_change = green_error[(green_error['member_yn']=='Y') & (green_error['green']==0)]['key']
green_remove = green_error[(green_error['member_yn']=='N') & (green_error['green']==0)]['key']

# 골드 할인
gold_error = merge_df[merge_df['discount_type'].str.contains('골드')]
gold_change = gold_error[(gold_error['member_yn']=='Y') & (gold_error['gold']==0)]['key']
gold_remove = gold_error[(gold_error['member_yn']=='N') & (gold_error['gold']==0)]['key']

In [107]:
# 삭제하기
remove_list = list(blue_remove) + list(green_remove) + list(gold_remove)
# 위 3개 케이스 제거
merge_df = merge_df[~merge_df['key'].isin(remove_list)]
merge_df.shape

(936187, 29)

In [108]:
# 값 바꿔주기
merge_df['r_blue'] = np.where(merge_df['key'].isin(blue_change), 1, merge_df['blue'])
merge_df['r_green'] = np.where(merge_df['key'].isin(green_change), 1, merge_df['green'])
merge_df['r_gold'] = np.where(merge_df['key'].isin(gold_change), 1, merge_df['gold'])

In [109]:
merge_df.drop(['blue', 'green', 'gold'], axis=1, inplace=True)
merge_df.rename(columns={'r_blue':'blue', 'r_green':'green', 'r_gold':'gold'}, inplace=True)

In [111]:
merge_df.shape

(936187, 29)

In [113]:
merge_df.columns

Index(['age', 'gender', 'tran_date', 'tran_time', 'play_date', 'play_st_time',
       'seat', 'price', 'ticket_cancel', 'discount_type', 'pre_open_date',
       'open_date', 'genre', 'running_time', 'intermission', 'member_yn',
       'new_code', 'key', 'pre_ticketing', 'general', 'sac', 'noble',
       'discount_rate', 'real_price', 'discount_cat', 'all_mem_cnt', 'blue',
       'green', 'gold'],
      dtype='object')

In [114]:
merge_df[(merge_df['green']==1) & (merge_df['member_yn']=='N')]

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,general,sac,noble,discount_rate,real_price,discount_cat,all_mem_cnt,blue,green,gold


# 취소표

In [116]:
merge_df.shape

(936187, 29)

In [117]:
group_single = merge_df.groupby(['new_code', 'seat'])
single_result = group_single.filter(lambda x: x['tran_time'].nunique() < 2)[['key', 'new_code', 'seat', 'tran_date', 'tran_time', 'ticket_cancel', 'discount_type']]
sort_single_df = single_result.sort_values(by=['new_code','seat', 'tran_date', 'tran_time'])
sort_single_df

,key,new_code,seat,tran_date,tran_time,ticket_cancel,discount_type
469463,549406,0,1층 A블록10열 1,20181118,15:52,0,기획사판매
812321,951361,0,1층 A블록10열 10,20181105,09:44,0,초대권
591785,692788,0,1층 A블록10열 11,20181105,09:44,0,초대권
65538,76662,0,1층 A블록10열 2,20181118,15:52,0,기획사판매
181838,212577,0,1층 A블록10열 3,20181118,15:52,0,기획사판매
...,...,...,...,...,...,...,...
613473,718348,750,합창석 H블록3열 15,20230701,23:01,0,골드회원 할인10%
37395,43724,750,합창석 H블록3열 2,20230606,21:31,0,그린회원 할인5%
238285,278659,750,합창석 H블록3열 21,20230517,22:40,0,일반
883338,1034381,750,합창석 H블록3열 23,20230409,14:04,0,골드회원 할인10%


In [118]:
grouped = merge_df.groupby(['new_code', 'seat'])
result = grouped.filter(lambda x: x['tran_time'].nunique() >= 2)[['key', 'new_code', 'seat', 'tran_date', 'tran_time', 'ticket_cancel', 'discount_type']]
sort_df = result.sort_values(by=['new_code','seat', 'tran_date', 'tran_time'])
sort_df

,key,new_code,seat,tran_date,tran_time,ticket_cancel,discount_type
92638,108267,0,1층 B블록10열 10,20181112,11:08,2,기획사판매
457554,535468,0,1층 B블록10열 10,20181112,11:18,0,기획사판매
19148,22395,0,1층 B블록10열 11,20181112,11:08,2,기획사판매
378567,442881,0,1층 B블록10열 11,20181112,11:18,0,기획사판매
211923,247834,0,1층 B블록10열 3,20181112,11:08,2,기획사판매
...,...,...,...,...,...,...,...
204794,239509,750,합창석 H블록3열 20,20230522,23:39,0,일반
770004,901783,750,합창석 H블록3열 20,20230523,00:06,2,그린회원 할인5%
661285,774409,750,합창석 H블록3열 20,20230703,17:16,0,싹틔우미 할인40%
371889,434996,750,합창석 H블록3열 22,20230409,14:06,2,블루회원 할인5%


In [119]:
filtered_df = sort_df.reset_index(drop=True)

In [120]:
# seat과 new_code가 같으면 같은 groupkey를 부여
filtered_df['groupkey'] = (filtered_df['seat'].astype(str) + filtered_df['new_code'].astype(str)).astype('category').cat.codes
filtered_df

,key,new_code,seat,tran_date,tran_time,ticket_cancel,discount_type,groupkey
0,108267,0,1층 B블록10열 10,20181112,11:08,2,기획사판매,7932
1,535468,0,1층 B블록10열 10,20181112,11:18,0,기획사판매,7932
2,22395,0,1층 B블록10열 11,20181112,11:08,2,기획사판매,7995
3,442881,0,1층 B블록10열 11,20181112,11:18,0,기획사판매,7995
4,247834,0,1층 B블록10열 3,20181112,11:08,2,기획사판매,8136
...,...,...,...,...,...,...,...,...
301422,239509,750,합창석 H블록3열 20,20230522,23:39,0,일반,117967
301423,901783,750,합창석 H블록3열 20,20230523,00:06,2,그린회원 할인5%,117967
301424,774409,750,합창석 H블록3열 20,20230703,17:16,0,싹틔우미 할인40%,117967
301425,434996,750,합창석 H블록3열 22,20230409,14:06,2,블루회원 할인5%,118048


In [121]:
# 각 그룹에서 마지막 행만 남기고 나머지 행을 삭제합니다.
df =  filtered_df.groupby('groupkey').tail(1)
df

,key,new_code,seat,tran_date,tran_time,ticket_cancel,discount_type,groupkey
1,535468,0,1층 B블록10열 10,20181112,11:18,0,기획사판매,7932
3,442881,0,1층 B블록10열 11,20181112,11:18,0,기획사판매,7995
5,525968,0,1층 B블록10열 3,20181112,11:18,0,기획사판매,8136
7,958660,0,1층 B블록10열 4,20181112,11:18,0,기획사판매,8177
9,1056534,0,1층 B블록10열 5,20181112,11:18,0,기획사판매,8218
...,...,...,...,...,...,...,...,...
301415,325441,750,합창석 H블록2열 18,20230703,10:19,0,싹틔우미 할인40%,117295
301419,13791,750,합창석 H블록3열 1,20230614,22:24,2,그린회원 할인5%,117838
301421,67330,750,합창석 H블록3열 13,20230505,04:15,2,싹틔우미 할인40%,117724
301424,774409,750,합창석 H블록3열 20,20230703,17:16,0,싹틔우미 할인40%,117967


In [122]:
df.groupkey.nunique()

118694

In [123]:
# 살릴 행의 key 정보 담기
single_list = list(sort_single_df['key'].values) + list(df['key'].values)
len(single_list)

753454

In [124]:
# 위 3개 케이스 제거
merge_df = merge_df[merge_df['key'].isin(single_list)]
merge_df.shape

(753454, 29)

In [125]:
# 전처리 된 데이터 저장
merge_df.to_csv('../processed_v1.csv', index=False)